In [1]:
from typing import Union, List, Dict

from backtesting.data import OrderStatus, OrderRequest
from backtesting.strategy import Strategy
from utils.data import Trade, OrderBook
from backtesting import readers, backtest
from backtesting.output import StorageOutput
from backtesting.readers import OrderbookReader

from metrics.filters import Filters
from metrics.metrics import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
class GatlingMM(Strategy):
  def __init__(self, side_volume=30000):
    super().__init__()
    self.side_volume = side_volume
    self.volumes_left = {} # add cancellation policy

  def define_orders(self, row: Union[Trade, OrderBook],
                    statuses: List[OrderStatus],
                    memory: Dict[str, Union[Trade, OrderBook]]) -> List[OrderRequest]:
    if self.balance.get(row.symbol, None) is None:
      if isinstance(row, OrderBook) and self.balance.get(row.symbol, None) is None:
        # Initialize first orders
        ask_volume = min(self._get_allowed_volume(row.symbol, memory, 'ask'), self.side_volume)
        self.volumes_left[(row.symbol, 'ask')] = self.side_volume - ask_volume
        ask_order = OrderRequest.create_ask(row.ask_prices[0], ask_volume, row.symbol, row.timestamp)

        bid_volume = min(self._get_allowed_volume(row.symbol, memory, 'bid'), self.side_volume)
        self.volumes_left[(row.symbol, 'bid')] = self.side_volume - bid_volume
        bid_order = OrderRequest.create_bid(row.bid_prices[0], bid_volume, row.symbol, row.timestamp)

        return [ask_order, bid_order]
    elif self.balance.get(row.symbol, None) is not None:
      orders = []
      for status in statuses:
        order: OrderRequest = self.active_orders[status.id]

        if status.status == 'finished':
          self.volumes_left[(order.symbol, order.side)] += order.volume - order.volume_filled
        elif status.status == 'partial':
          self.volumes_left[(order.symbol, order.side)] += status.volume

        volume = min(self.volumes_left[(order.symbol, order.side)], self._get_allowed_volume(order.symbol, memory, order.side))
        if volume > 0:
          self.volumes_left[(order.symbol, order.side)] -= volume

          if order.side == 'bid':
            price = memory[('orderbook', order.symbol)].bid_prices[0]
          else:
            price = memory[('orderbook', order.symbol)].ask_prices[0]

          neworder = OrderRequest.create(price, volume, order.symbol, order.side, row.timestamp)
          orders.append(neworder)

      return orders
    return []

In [8]:
reader = OrderbookReader('../tests/resources/huge_dataset/orderbook_10_03_20.csv.gz', 
                         '../tests/resources/huge_dataset/trades_10_03_20.csv.gz',
                         stop_after=300000, depth_to_load=5)

output = StorageOutput([], [])
output.balances = []
strategy = GatlingMM()
strategy.balance_listener = lambda b, t: output.balances.append((t, b))

backtester = backtest.Backtest(reader, strategy, output=output, delay=300)


2020-03-25 00:08:10,628 - <backtest> - INFO - Initialized <Backtest with reader=<orderbook-reader on orderbook_file=../tests/resources/huge_dataset/orderbook_10_03_20.csv.gz, trades_file=../tests/resources/huge_dataset/trades_10_03_20.csv.gz, batch_nrows=10000>>


In [9]:
from utils import logger
logger.set_fmt_string("%(name)s: %(message)s")

In [10]:
backtester.run()

2020-03-25 00:08:12,494 - <backtest> - INFO - Backtest initialize run
2020-03-25 00:08:12,501 - <Strategy> - INFO - New order: <order-request id=27, command=new, symbol=XBTUSD, side=ask, volume=30000, price=7823.0>
2020-03-25 00:08:12,503 - <Strategy> - INFO - New order: <order-request id=28, command=new, symbol=XBTUSD, side=bid, volume=30000, price=7822.5>
2020-03-25 00:08:12,507 - <Strategy> - INFO - New order: <order-request id=29, command=new, symbol=ETHUSD, side=ask, volume=30000, price=199.05>
2020-03-25 00:08:12,508 - <Strategy> - INFO - New order: <order-request id=30, command=new, symbol=ETHUSD, side=bid, volume=12138.449999999999, price=199.0>
2020-03-25 00:08:15,675 - <Strategy> - INFO - Received status: <order-status id=28, status=partial, volume=2038, at=2020-03-10 18:25:35.703000>
2020-03-25 00:08:15,676 - <Strategy> - INFO - Received status: <order-status id=28, status=partial, volume=3130, at=2020-03-10 18:25:35.703000>
2020-03-25 00:08:15,679 - <Strategy> - INFO - Rece

2020-03-25 00:08:17,614 - <Strategy> - INFO - Received status: <order-status id=35, status=partial, volume=3940, at=2020-03-10 18:25:37.255000>
2020-03-25 00:08:17,616 - <Strategy> - INFO - Received status: <order-status id=40, status=partial, volume=233, at=2020-03-10 18:25:37.255000>
2020-03-25 00:08:17,617 - <Strategy> - INFO - Received status: <order-status id=35, status=partial, volume=4093, at=2020-03-10 18:25:37.255000>
2020-03-25 00:08:17,620 - <Strategy> - INFO - Received status: <order-status id=40, status=partial, volume=539, at=2020-03-10 18:25:37.255000>
2020-03-25 00:08:17,627 - <Strategy> - INFO - Received status: <order-status id=35, status=partial, volume=4399, at=2020-03-10 18:25:37.255000>
2020-03-25 00:08:17,631 - <Strategy> - INFO - Received status: <order-status id=40, status=partial, volume=2939, at=2020-03-10 18:25:37.255000>
2020-03-25 00:08:17,633 - <Strategy> - INFO - Received status: <order-status id=35, status=partial, volume=6798, at=2020-03-10 18:25:37.25

2020-03-25 00:08:17,877 - <Strategy> - INFO - New order: <order-request id=72, command=new, symbol=XBTUSD, side=bid, volume=4235, price=7817.0>
2020-03-25 00:08:17,880 - <Strategy> - INFO - New order: <order-request id=73, command=new, symbol=XBTUSD, side=bid, volume=8094, price=7817.0>
2020-03-25 00:08:17,882 - <Strategy> - INFO - New order: <order-request id=74, command=new, symbol=XBTUSD, side=bid, volume=4259, price=7817.0>
2020-03-25 00:08:17,884 - <Strategy> - INFO - New order: <order-request id=75, command=new, symbol=XBTUSD, side=bid, volume=8118, price=7817.0>
2020-03-25 00:08:17,886 - <Strategy> - INFO - New order: <order-request id=76, command=new, symbol=XBTUSD, side=bid, volume=4289, price=7817.0>
2020-03-25 00:08:17,889 - <Strategy> - INFO - New order: <order-request id=77, command=new, symbol=XBTUSD, side=bid, volume=8148, price=7817.0>
2020-03-25 00:08:17,890 - <Strategy> - INFO - New order: <order-request id=78, command=new, symbol=XBTUSD, side=bid, volume=4539, price=

2020-03-25 00:08:20,275 - <Strategy> - INFO - Received status: <order-status id=53, status=partial, volume=585, at=2020-03-10 18:25:38.521000>
2020-03-25 00:08:20,277 - <Strategy> - INFO - Received status: <order-status id=42, status=partial, volume=1635, at=2020-03-10 18:25:38.521000>
2020-03-25 00:08:20,282 - <Strategy> - INFO - Received status: <order-status id=37, status=partial, volume=5495, at=2020-03-10 18:25:38.521000>
2020-03-25 00:08:20,294 - <Strategy> - INFO - New order: <order-request id=109, command=new, symbol=XBTUSD, side=bid, volume=8529, price=7816.5>
2020-03-25 00:08:20,305 - <Strategy> - INFO - New order: <order-request id=110, command=new, symbol=XBTUSD, side=bid, volume=465, price=7816.5>
2020-03-25 00:08:20,307 - <Strategy> - INFO - New order: <order-request id=111, command=new, symbol=XBTUSD, side=bid, volume=4325, price=7816.5>
2020-03-25 00:08:20,310 - <Strategy> - INFO - New order: <order-request id=112, command=new, symbol=XBTUSD, side=bid, volume=585, price

2020-03-25 00:08:21,205 - <Strategy> - INFO - Received status: <order-status id=56, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
2020-03-25 00:08:21,207 - <Strategy> - INFO - Received status: <order-status id=57, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
2020-03-25 00:08:21,209 - <Strategy> - INFO - Received status: <order-status id=58, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
2020-03-25 00:08:21,210 - <Strategy> - INFO - Received status: <order-status id=59, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
2020-03-25 00:08:21,214 - <Strategy> - INFO - Received status: <order-status id=60, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
2020-03-25 00:08:21,216 - <Strategy> - INFO - Received status: <order-status id=61, status=partial, volume=1352, at=2020-03-10 18:25:38.891000>
2020-03-25 00:08:21,218 - <Strategy> - INFO - Received status: <order-status id=46, status=finished, volume=-1, at=2020-03-10 18:25:38.891000

...

2020-03-25 00:08:31,970 - <Strategy> - INFO - Received status: <order-status id=163, status=finished, volume=-1, at=2020-03-10 18:25:51.915000>
2020-03-25 00:08:31,972 - <Strategy> - INFO - Received status: <order-status id=164, status=finished, volume=-1, at=2020-03-10 18:25:51.915000>
2020-03-25 00:08:31,975 - <Strategy> - INFO - Received status: <order-status id=165, status=finished, volume=-1, at=2020-03-10 18:25:51.915000>
2020-03-25 00:08:31,977 - <Strategy> - INFO - Received status: <order-status id=166, status=finished, volume=-1, at=2020-03-10 18:25:51.915000>
2020-03-25 00:08:31,979 - <Strategy> - INFO - Received status: <order-status id=167, status=finished, volume=-1, at=2020-03-10 18:25:51.915000>
2020-03-25 00:08:31,984 - <Strategy> - INFO - Received status: <order-status id=168, status=finished, volume=-1, at=2020-03-10 18:25:51.915000>
2020-03-25 00:08:31,987 - <Strategy> - INFO - Received status: <order-status id=169, status=finished, volume=-1, at=2020-03-10 18:25:51.

2020-03-25 00:08:32,078 - <Strategy> - INFO - New order: <order-request id=311, command=new, symbol=XBTUSD, side=bid, volume=50, price=7814.0>
2020-03-25 00:08:32,080 - <Strategy> - INFO - New order: <order-request id=312, command=new, symbol=XBTUSD, side=bid, volume=456, price=7814.0>
2020-03-25 00:08:32,082 - <Strategy> - INFO - New order: <order-request id=313, command=new, symbol=XBTUSD, side=bid, volume=64, price=7814.0>
2020-03-25 00:08:32,084 - <Strategy> - INFO - New order: <order-request id=314, command=new, symbol=XBTUSD, side=bid, volume=2046, price=7814.0>
2020-03-25 00:08:32,085 - <Strategy> - INFO - New order: <order-request id=315, command=new, symbol=XBTUSD, side=bid, volume=508, price=7814.0>
2020-03-25 00:08:32,087 - <Strategy> - INFO - New order: <order-request id=316, command=new, symbol=XBTUSD, side=bid, volume=2246, price=7814.0>
2020-03-25 00:08:32,088 - <Strategy> - INFO - New order: <order-request id=317, command=new, symbol=XBTUSD, side=bid, volume=707, price=

2020-03-25 00:08:32,753 - <Strategy> - INFO - Received status: <order-status id=196, status=partial, volume=1220, at=2020-03-10 18:25:53.234000>
2020-03-25 00:08:32,754 - <Strategy> - INFO - Received status: <order-status id=194, status=partial, volume=1633, at=2020-03-10 18:25:53.234000>
2020-03-25 00:08:32,756 - <Strategy> - INFO - Received status: <order-status id=196, status=partial, volume=1254, at=2020-03-10 18:25:53.234000>
2020-03-25 00:08:32,757 - <Strategy> - INFO - Received status: <order-status id=194, status=partial, volume=1655, at=2020-03-10 18:25:53.234000>
2020-03-25 00:08:32,759 - <Strategy> - INFO - Received status: <order-status id=196, status=partial, volume=1277, at=2020-03-10 18:25:53.234000>
2020-03-25 00:08:32,760 - <Strategy> - INFO - Received status: <order-status id=194, status=partial, volume=1686, at=2020-03-10 18:25:53.234000>
2020-03-25 00:08:32,761 - <Strategy> - INFO - Received status: <order-status id=196, status=partial, volume=1308, at=2020-03-10 18

2020-03-25 00:08:33,695 - <Strategy> - INFO - New order: <order-request id=369, command=new, symbol=XBTUSD, side=bid, volume=5738, price=7814.0>
2020-03-25 00:08:33,697 - <Strategy> - INFO - New order: <order-request id=370, command=new, symbol=XBTUSD, side=bid, volume=6, price=7814.0>
2020-03-25 00:08:33,700 - <Strategy> - INFO - New order: <order-request id=371, command=new, symbol=XBTUSD, side=bid, volume=5782, price=7814.0>
2020-03-25 00:08:33,702 - <Strategy> - INFO - New order: <order-request id=372, command=new, symbol=XBTUSD, side=bid, volume=50, price=7814.0>
2020-03-25 00:08:33,704 - <Strategy> - INFO - New order: <order-request id=373, command=new, symbol=XBTUSD, side=bid, volume=706, price=7814.0>
2020-03-25 00:08:33,708 - <Strategy> - INFO - New order: <order-request id=374, command=new, symbol=XBTUSD, side=bid, volume=2654, price=7814.0>
2020-03-25 00:08:34,458 - <Strategy> - INFO - Received status: <order-status id=308, status=finished, volume=-1, at=2020-03-10 18:26:04.

KeyboardInterrupt: 

In [11]:
strategy.balance

defaultdict(<function backtesting.strategy.Strategy.__init__.<locals>.<lambda>>,
            {'ETHUSD': -63.24588020006401,
             'USD': -83611.99999999994,
             'XBTUSD': 88.1914275752483})

In [18]:
backtester.memory[('orderbook', 'XBTUSD')].timestamp

datetime.datetime(2020, 3, 10, 18, 26, 50, 807000)